In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb, platform, warnings
import gradio 
from datasets import load_dataset
from trl import SFTTrainer

/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [16]:
import torch
torch.cuda.empty_cache()

In [17]:
 print('메모리 사용량/총 메모리: ' + str(torch.cuda.memory_allocated('cuda:0')/1024**3 ) + 'GB / ' + str(torch.cuda.memory_reserved('cuda:0')/1024**3 ) + 'GB')

메모리 사용량/총 메모리: 29.900827884674072GB / 30.8203125GB


In [4]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: NVIDIA RTX A6000
Compute Capability: (8, 6)
Total Memory: 51041271808 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.2.0-34-generic
Python Version: 3.9.12


In [5]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ········
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate throu

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

config = PeftConfig.from_pretrained('/data/dhk/chat_c/')
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, '/data/dhk/chat_c/')
model = model.to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_featur

In [9]:
print(config)

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='Minirecord/Mini_synatra_7b_02', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=16, target_modules={'o_proj', 'k_proj', 'q_proj', 'v_proj', 'gate_proj', 'up_proj'}, lora_alpha=8, lora_dropout=0.1, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={})


In [10]:
# 예시 코드
example_sentence = "이것은 테스트 문장입니다."
tokenized_output = tokenizer.tokenize(example_sentence)
print(tokenized_output)

# 토크나이저와 모델의 임베딩 레이어 크기 비교
print(len(tokenizer.vocab))  # 토크나이저 단어장 크기

['▁', '이', '것', '은', '▁', '테', '스', '트', '▁', '문', '장', '입', '니', '다', '.']
32002


In [11]:
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
                (base_layer): Linear(in_featur

In [12]:
def stream(user_prompt):
    runtimeFlag = "cuda:0"
    system_prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    B_INST, E_INST = "### Instruction:", "### Response:"

    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    inputs = tokenizer([prompt], return_tensors="pt").to(runtimeFlag)

    # Generate response
    outputs = model.generate(**inputs, max_new_tokens=1000, temperature=1.0, length_penalty=0.2, do_sample=True, num_beams=10)

    # Decode and print the generated response
    print(tokenizer.decode(outputs[0]))

stream("나 오늘 슬퍼")

/home/dhkim/anaconda3/envs/llama_dhk/lib/python3.9/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> Below is an instruction that describes a task. Write a response that appropriately completes the request.### Instruction:나 오늘 슬퍼### Response: 슬픈 일이 있었나봐요. 제가 도와드릴게요.</s>


In [ ]:
stream("나 오늘 떡볶이 먹어서 기뻐")